In [ ]:

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *
spark = SparkSession.builder \
.master("local[*]") \
.config("spark.driver.memory", "6g") \
.getOrCreate()
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/21 07:10:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
! wget https://github.com/erkansirin78/datasets/raw/refs/heads/master/yellow_tripdata_partitioned_by_day/year=2024/month=10/day=11/part-00135-5bd1507a-125d-4ec6-8794-3ed02e45a45d.c000.snappy.parquet
--2025-06-21 07:14:09--  https://github.com/erkansirin78/datasets/raw/refs/heads/master/yellow_tripdata_partitioned_by_day/year=2024/month=10/day=11/part-00135-5bd1507a-125d-4ec6-8794-3ed02e45a45d.c000.snappy.parquet
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/erkansirin78/datasets/refs/heads/master/yellow_tripdata_partitioned_by_day/year%3D2024/month%3D10/day%3D11/part-00135-5bd1507a-125d-4ec6-8794-3ed02e45a45d.c000.snappy.parquet [following]
--2025-06-21 07:14:10--  https://raw.githubusercontent.com/erkansirin78/datasets/refs/heads/master/yellow_tripdata_partitioned_by_day/year%3D2024/month%3D10/day%3D11/part-00135-5bd1507a-125d-4ec6-8794-3ed02e45a45d.c000.snappy.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 355218 (347K) [application/octet-stream]
Saving to: ‘part-00135-5bd1507a-125d-4ec6-8794-3ed02e45a45d.c000.snappy.parquet’

part-00135-5bd1507a 100%[===================>] 346.89K  --.-KB/s    in 0.04s   

2025-06-21 07:14:10 (9.02 MB/s) - ‘part-00135-5bd1507a-125d-4ec6-8794-3ed02e45a45d.c000.snappy.parquet’ saved [355218/355218]

df = spark.read.format("parquet").load("file:///taxi")
df.count()
128717
df.limit(5).toPandas()
VendorID	tpep_pickup_datetime	tpep_dropoff_datetime	passenger_count	trip_distance	RatecodeID	store_and_fwd_flag	PULocationID	DOLocationID	payment_type	fare_amount	extra	mta_tax	tip_amount	tolls_amount	improvement_surcharge	total_amount	congestion_surcharge	Airport_fee
0	2	2024-10-11 20:08:40	2024-10-11 20:14:29	1.0	1.11	1.0	N	237	162	2	7.9	1.0	0.5	0.00	0.00	1.0	12.90	2.5	0.00
1	2	2024-10-11 20:25:42	2024-10-11 20:35:23	1.0	1.95	1.0	N	163	236	1	11.4	1.0	0.5	1.64	0.00	1.0	18.04	2.5	0.00
2	2	2024-10-11 20:38:41	2024-10-11 20:50:56	1.0	1.56	1.0	N	43	262	1	12.8	1.0	0.5	3.56	0.00	1.0	21.36	2.5	0.00
3	2	2024-10-10 20:56:46	2024-10-11 20:48:01	2.0	2.50	1.0	N	161	158	1	20.5	1.0	0.5	0.00	0.00	1.0	25.50	2.5	0.00
4	2	2024-10-10 21:48:05	2024-10-11 20:53:38	1.0	10.59	1.0	N	138	125	1	45.7	6.0	0.5	6.44	6.94	1.0	70.83	2.5	1.75
df.printSchema()
root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)

TripID
PickupDateTimeID
DropOffDateTimeID
PULocationID
DOLocationID
RateCodeID
PaymentTypeID
PassengerCount
TripDistance
FareAmount
TipAmount
TollsAmount
AirportAmount
TotalAmount
Extra
MTATax
CongestionSurcharge
Transform timestamps
df2 = df.withColumn("PickupDateTimeID",  F.date_format("tpep_pickup_datetime", "yyyyMMddHHmm").cast("long")) \
.withColumn("DropOffDateTimeID",  F.date_format("tpep_dropoff_datetime", "yyyyMMddHHmm").cast("long"))
df2.columns
['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee',
 'PickupDateTimeID',
 'DropOffDateTimeID']
df3 = df2.select(
    F.monotonically_increasing_id().alias("TripID"),
    F.col("PickupDateTimeID"),
    F.col("DropOffDateTimeID"),
    F.col("PULocationID"),
    F.col("DOLocationID"),
    F.col("RatecodeID").alias("RateCodeID"),
    F.col("payment_type").alias("PaymentTypeID"),
    F.col("passenger_count").alias("PassengerCount"),
    F.col("trip_distance").alias("TripDistance"),
    F.col("fare_amount").alias("FareAmount"),
    F.col("tip_amount").alias("TipAmount"),
    F.col("tolls_amount").alias("TollsAmount"),
    F.col("Airport_fee").alias("AirportAmount"),
    F.col("total_amount").alias("TotalAmount"),
    F.col("extra").alias("Extra"),
    F.col("mta_tax").alias("MTATax"),
    F.col("congestion_surcharge").alias("CongestionSurcharge")
)
df3.limit(5).toPandas()
TripID	PickupDateTimeID	DropOffDateTimeID	PULocationID	DOLocationID	RateCodeID	PaymentTypeID	PassengerCount	TripDistance	FareAmount	TipAmount	TollsAmount	AirportAmount	TotalAmount	Extra	MTATax	CongestionSurcharge
0	0	202410112008	202410112014	237	162	1.0	2	1.0	1.11	7.9	0.00	0.00	0.00	12.90	1.0	0.5	2.5
1	1	202410112025	202410112035	163	236	1.0	1	1.0	1.95	11.4	1.64	0.00	0.00	18.04	1.0	0.5	2.5
2	2	202410112038	202410112050	43	262	1.0	1	1.0	1.56	12.8	3.56	0.00	0.00	21.36	1.0	0.5	2.5
3	3	202410102056	202410112048	161	158	1.0	1	2.0	2.50	20.5	0.00	0.00	0.00	25.50	1.0	0.5	2.5
4	4	202410102148	202410112053	138	125	1.0	1	1.0	10.59	45.7	6.44	6.94	1.75	70.83	6.0	0.5	2.5
 